In [45]:
import smartdata as smdt

In [46]:
import smartdata
from smartdata.utils.common import print_attributes as pa
from smartdata.utils.dframe import display_full_df as ddff
from smartdata.utils.common import to_namedtuple
from smartdata.utils.file import read_from_json
from smartdata import SmartData


In [1]:
import gmt.arc.sync as arc

In [1]:
from gmt.dna.datagateway.data_adapter_factory import DataAdapterFactory

In [ ]:
def query_arc_alchemy(sql):
    # ODBC connection string
    _url ='mssql+pyodbc:///?odbc_connect=DATABASE%3DARCTIMESERIES%3BDRIVER%3D%7BSQL+Server+Native+Client+11.0%7D%3BSERVER%3DARCSQL%3BTrace%3D0%3BTrusted_Connection%3DYes'
    engine =sa.create_engine(_url, echo=False, convert_unicode=True)
    with engine.connect() as connection:
        result =connection.execute(sql)
        data =result.fetchall()
        column_names =result.keys()
        dataframe = pd.DataFrame.from_records(data, columns=column_names)

        return dataframe

#This allows each loop of the netbacks to be added to the same dataframe in in line 263
Netback2 = pd.DataFrame()
Variables_Prices2 = pd.DataFrame()

# ===============================================
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int((date2 - date1).days) + 1):
        yield date1 + timedelta(n)


start_dt = date(2021, 1, 14)
end_dt = date(2021, 1, 14)

weekend = [6, 7]
for dt in daterange(start_dt, end_dt):
    if dt.isoweekday() in weekend:
        continue

    # print(dt.strftime("%Y-%m-%d"))

    # ===================================================
    #THIS ALLOWS THE CONTRACTS TO BE ROLLED ON 20TH IN-LINE WITH JKM FM AND ALSO PROVIDES THE SQL FOR THE REST OF THE QUERY
    # https://stackoverflow.com/questions/546321/how-do-i-calculate-the-date-six-months-from-the-current-date-using-the-datetime
    today = dt.day
    if today > 20:   
        from_date = dt + pd.DateOffset(months=2)
        to_date = dt + pd.DateOffset(months=2 ,years=3)
    else:
        from_date = dt + pd.DateOffset(months=1)
        to_date = dt + pd.DateOffset(months=1 ,years=3)
    query_str_1 = f"""
SELECT
  d.[CurveName],
  datefromparts(year(min(d.PointDate)), month(min(d.PointDate)), 1) as PointDate,
  AVG(d.[PointValue]) as PointValue
FROM [arc_ts_data].[PriceCurveData] d
WHERE d.EffectiveDate = '{dt.strftime("%Y-%m-%d")}' and [CurveName] in 
('G_H_NBP.USD', 'G_H_Zeebrugge-Hub.Pnc','LNG_JKM_M.USD', 'G_H_TRS.EUR', 
'LNG_DES_ME_W_INDIA_M.USD', 'LNG_H_DES_Med_MID_M.USD', 'LNG_H_DES_JKM_MID_M.USD', 
'LNG_H_DES_NWE_MID_M.USD', 'G_H_TTF.EUR','G_H_CEGH.EUR', 'G_H_AOC.EUR', 'G_H_PEG-Nord.EUR', 
'G_H_PSV.EUR','G_NYM_HH_M.USD' , 'LNG_H_DES_Lat_Am_MID_M.USD', 'FX_curve_cross_EUR.USD', 
'FX_curve_input_USD.GBP', 'FREIGHT_H_LNG_M_RolledOver.USD')  
and d.PointDate >= '{from_date.strftime("%Y-%m-01")}' and d.PointDate < '{to_date.strftime("%Y-%m-01")}'
group by d.CurveId, d.CurveName, d.EffectiveDate, 12 * year(d.PointDate) + month(d.PointDate)
order by d.CurveID
"""

    Prices = query_arc_alchemy(query_str_1)
    # https://stackoverflow.com/questions/56315468/replace-comma-and-dot-in-pandas/56315638
    Prices['CurveName'] = Prices['CurveName'].str.replace('.', '_')
    Prices['PointDate_Origin'] = pd.to_datetime(Prices['PointDate'])
    Prices['EffectiveDate'] = dt
    Prices['EffectiveDate'] = pd.to_datetime(Prices['EffectiveDate'])
    Prices['PointDate_Destination'] = Prices['PointDate_Origin']   
    columns = ['CurveName', 'PointValue', 'PointDate_Origin', 'EffectiveDate','PointDate_Destination']
    Prices = Prices[columns]
    
    #Prices.rename(columns={'PointValue':'PointValue_Destination'}, inplace=True)
    # =======================================================================================
    # https://stackoverflow.com/questions/43055050/python-pandas-if-column-string-contains-word-flag
    # This is to look up a partial string in a column.
    # Prices['Conversion'] = np.where(Prices['CurveName'].str.contains("FX_curve_cross_EUR_USD", case=False, na=False), 'T', '')

    mapper = Prices.query('CurveName == "FX_curve_cross_EUR_USD"').set_index(['PointDate_Origin'])['PointValue']
    Prices['FX_curve_cross_EUR_USD'] = Prices['PointDate_Origin'].map(mapper)
    

    # Converts the Curves that are in EUR into USD.  #CHECK WHETHER YOU COULD JUST SEARCH FOR THE USD VERSION OF THE CURVE IN THE DATABASE
    # Prices['PointValue'] at the end of these two lines of code tells python to just keep the current pointvalue if the curvename does not have _EUR in it.
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("_EUR", case=False),
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') / Prices['FX_curve_cross_EUR_USD'].apply(pd.to_numeric, downcast='float') / 3.412 * 0.95,
                                    Prices['PointValue'])
    # ===============================================================================================

    mapper = Prices.query('CurveName == "FX_curve_input_USD_GBP"').set_index(['PointDate_Origin'])['PointValue']
    Prices['FX_curve_input_USD_GBP'] = Prices['PointDate_Origin'].map(mapper)

    # Converts the Curves that are in pnc into USD.  #CHECK WHETHER YOU COULD JUST SEARCH FOR THE USD VERSION OF THE CURVE IN THE DATABASE
    # Prices['PointValue'] at the end of these two lines of code tells python to just keep the current pointvalue if the curvename does not have _EUR in it.
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("_Pnc", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') / Prices['FX_curve_input_USD_GBP'].apply(pd.to_numeric, downcast='float') / 10, 
                                    Prices['PointValue'])
    
    #INCREASES THE HH PRICE BY 15% IN-LINE WITH CHENIERE CONTRACTS
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_NYM_HH_M_USD", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 1.15, 
                                    Prices['PointValue'])
    Prices['CurveName'] = np.where((Prices['CurveName'] == 'G_NYM_HH_M_USD'),'G_NYM_HH_M_115%_USD',Prices['CurveName'])
    
     #INCREASES THE NBP PRICE BY 15% IN-LINE WITH CHENIERE CONTRACTS
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_NBP_USD", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0.95, 
                                    Prices['PointValue'])
    Prices['CurveName'] = np.where((Prices['CurveName'] == 'G_H_NBP_USD'),'G_H_NBP_95%_USD',Prices['CurveName'])
    
         #INCREASES THE NBP PRICE BY 15% IN-LINE WITH CHENIERE CONTRACTS
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_TTF_EUR", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0.95, 
                                    Prices['PointValue'])
    Prices['CurveName'] = np.where((Prices['CurveName'] == 'G_H_TTF_EUR'),'G_H_TTF_95%_EUR',Prices['CurveName'])
    
     #DECREASES THE JKM PRICE BY 100% 
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_CEGH_EUR", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0, 
                                    Prices['PointValue'])
    Prices['CurveName'] = np.where((Prices['CurveName'] == 'G_H_CEGH_EUR'),'LNG_JKM_M_0%_USD',Prices['CurveName'])

    # ==============================================================================
    Prices['FX_curve_cross_EUR_USD'] = Prices['FX_curve_cross_EUR_USD'].fillna(method='ffill')
    Prices['FX_curve_input_USD_GBP'] = Prices['FX_curve_input_USD_GBP'].fillna(method='ffill')

    # Looks up the freight costs in prices then assigns it to a new column shipping cost in variables and then calculates the full shipping costs
    mapper = Prices.query('CurveName == "FREIGHT_H_LNG_M_RolledOver_USD"').set_index(['PointDate_Origin'])['PointValue']
    PriceVariables = Variables.copy()
    # https://stackoverflow.com/questions/34915828/pandas-date-range-to-generate-monthly-data-at-beginning-of-the-month
    date_rng = pd.date_range(start=dt, freq='MS', periods=12, name='Dates')
    combining_date = []
    for d in date_rng:
        PriceVariables['PointDate_Origin'] = d
        combining_date.append(PriceVariables.copy())

    PriceVariables = pd.concat(combining_date)
    #===========================================================================   
    #CALCULATES SHIPPING COST, WORKS OUT IF TRIP CROSSES TWO MONTHS, CONVERTS DATA TO FLOAt & CREATES OUPUT VARIABLE
    #This line matches the point date with the corresponding shipping cost and if the days travel is more than 30 days then it takes the average of the two months
    PriceVariables['Shipping Cost'] = np.where(PriceVariables['Days Travel (Return - inc. time at port)'] > 30, 
                  ((PriceVariables['PointDate_Origin'] + pd.DateOffset(months=1)).map(mapper) + PriceVariables['PointDate_Origin'].map(mapper)) / 2, 
                  PriceVariables['PointDate_Origin'].map(mapper))
    
    # convert all the variables into float from decimal
    PriceVariables[['Regas Costs', 'Port Charges', 'Boil-off', 'Delivered Volume', 'Shipping Cost']] = PriceVariables[
        ['Regas Costs', 'Port Charges', 'Boil-off', 'Delivered Volume', 'Shipping Cost']].apply(pd.to_numeric,
                                                                                                downcast='float')
    # Variables['Shipping Cost'] = Variables['Shipping Cost'].apply(pd.to_numeric, downcast='float')

    # Shipping Cost Output is the first cost calcualtion
    PriceVariables['ShippingCostOutput'] = (
        PriceVariables['Shipping Cost'] *
        PriceVariables['Days Travel (Return - inc. time at port)'] /
        Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]
    )
    #===========================================================================
    # Maps Price Curve to a Terminal Location so that Boil off can be calculated
    conditions3 = [
        (PriceVariables['TerminalTo'] == "Argentina (B'Blanca)"),
        (PriceVariables['TerminalTo'] == 'Belgium (Zee)'),
        (PriceVariables['TerminalTo'] == 'China (Guandong)'),
        (PriceVariables['TerminalTo'] == 'France (Montoir)'),
        (PriceVariables['TerminalTo'] == 'India (Dhambol)'),
        (PriceVariables['TerminalTo'] == 'Italy (Rovigo)'),
        (PriceVariables['TerminalTo'] == 'Italy (Rovigo)(PSV)'),
        (PriceVariables['TerminalTo'] == 'Japan (Tokyo)'),
        (PriceVariables['TerminalTo'] == 'Middle East (UAE)'),
        (PriceVariables['TerminalTo'] == 'Netherlands (Gate)'),
        (PriceVariables['TerminalTo'] == 'Netherlands (Gate)(TTF)'),
        (PriceVariables['TerminalTo'] == 'Spain (Barcelona)'),
        (PriceVariables['TerminalTo'] == 'Spain (Barcelona)(PVB)'),
        (PriceVariables['TerminalTo'] == 'UK (Hook)'),
        (PriceVariables['TerminalTo'] == 'UK (Hook)(NBP)')]

    choices3 = ['LNG_H_DES_Lat_Am_MID_M_USD', 
                'G_H_Zeebrugge-Hub_Pnc', 
                'LNG_JKM_M_USD', 
                'G_H_PEG-Nord_EUR',
                'LNG_DES_ME_W_INDIA_M_USD',
                'LNG_H_DES_Med_MID_M_USD',
                "G_H_PSV_EUR", 
                'LNG_JKM_M_USD', 
                'LNG_DES_ME_W_INDIA_M_USD', 
                'LNG_H_DES_NWE_MID_M_USD',
                'G_H_TTF_95%_EUR',
                'LNG_H_DES_Med_MID_M_USD',
                'G_H_AOC_EUR', 
                'LNG_H_DES_NWE_MID_M_USD',
                'G_H_NBP_95%_USD']
    PriceVariables['CurveName'] = np.select(conditions3, choices3, default=9999999)
    #===========================================================================
    #===========================================================================
    # This combines the two datatables on PointDate and CurveName and PointValue
    Variables_Prices = PriceVariables.merge(Prices[['PointDate_Origin','CurveName','EffectiveDate']], how='inner', on=['PointDate_Origin', 'CurveName'])
    #===========================================================================
    #==================================================================================================
    # Maps Price Curve based on which terminal the Gas is liquified from.
    conditions4 = [
        (Variables_Prices['TerminalFrom'] == 'Sabine'),
        (Variables_Prices['TerminalFrom'] == 'Cove point'),
        (Variables_Prices['TerminalFrom'] == 'Nigeria'),
        (Variables_Prices['TerminalFrom'] == 'Angola'),
        (Variables_Prices['TerminalFrom'] == 'Qatar'),
        (Variables_Prices['TerminalFrom'] == 'Algeria'),
        (Variables_Prices['TerminalFrom'] == 'Yamal'),
        (Variables_Prices['TerminalFrom'] == 'Wheatstone'),
        (Variables_Prices['TerminalFrom'] == 'Gladstone'),
        (Variables_Prices['TerminalFrom'] == 'T&T')]

    choices4 = ['G_NYM_HH_M_115%_USD', 
                'G_NYM_HH_M_115%_USD',
                'LNG_JKM_M_0%_USD',
                'LNG_JKM_M_0%_USD',
                'LNG_JKM_M_0%_USD',
                'LNG_JKM_M_0%_USD',
                'LNG_JKM_M_0%_USD',
                'LNG_JKM_M_0%_USD',
                'LNG_JKM_M_0%_USD',
                'LNG_JKM_M_0%_USD']
    #In order to merge condtion 4, it needed to be merged on Variables_Prices and not PriceVariables.
    #PriceVariables['OriginPriceCurve'] = np.select(conditions4, choices4, default=9999999)
    Variables_Prices['OriginPriceCurve'] = np.select(conditions4, choices4, default=9999999)

    #This combines the two dataframes and has created x and y names which then get renamed below this line of code.
    Variables_Prices = Variables_Prices.merge(Prices[['PointDate_Origin', 'CurveName','PointValue']], how='inner', right_on=['PointDate_Origin', 'CurveName'],
                                   left_on=['PointDate_Origin', 'OriginPriceCurve'])

    
    Variables_Prices = Variables_Prices.drop(columns=['CurveName_y'])
   
    #Renamed the columns.
   #### IMPORTANT: I have renamed Pointvalue to PointValue_Destination, this is so that we can start adjusting Point value when the journey takes longer than 30 days.
    Variables_Prices.rename(columns={'CurveName_x':'CurveName_Destination','PointValue':'PointValue_Origin'}, inplace=True)
   
    #=============================================================================================
    #=============================================================================================
    #Adds PointDate_Destination which will change depending on whether the journey is longer than 30 days, this allows boil-off and the destination price to be used correctly.
    Variables_Prices['PointDate_Destination'] = Variables_Prices['PointDate_Origin']
    
    Variables_Prices['PointDate_Destination'] = np.where(Variables_Prices['Days Travel (Return - inc. time at port)'] > 30, 
                   (Variables_Prices['PointDate_Destination'] + pd.DateOffset(months=1)), 
                   Variables_Prices['PointDate_Destination'])
    
    Variables_Prices = Variables_Prices.merge(Prices[['PointDate_Destination','CurveName','PointValue']], right_on=['PointDate_Destination', 'CurveName'],
                                   left_on=['PointDate_Destination', 'CurveName_Destination'])
    Variables_Prices.rename(columns={'PointValue':'PointValue_Destination'}, inplace=True)
    Variables_Prices = Variables_Prices.drop(columns=['CurveName'])
    #=============================================================================================
    #=============================================================================================   
    #Converts the following columns into a float dtype
    Variables_Prices[['Boil-off', 'PointValue_Destination','PointValue_Origin']] = Variables_Prices[['Boil-off', 'PointValue_Destination', 'PointValue_Origin']].apply(pd.to_numeric,downcast='float')

    # Boil Off Output is the second cost calcualtion
    Variables_Prices['BoilOffOutput'] = (
            Variables_Prices['PointValue_Destination'] * Variables_Prices['Boil-off'] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0])

    # Fuel Oil Output is the third cost calcualtion
    # https://chrisalbon.com/python/data_wrangling/pandas_create_column_using_conditional/
    # Condition statement as a vector used np.where
    Variables_Prices['FuelOutput'] = np.where(Variables_Prices['TerminalFrom'] == 'Yamal', \
                                              ((Costs['OneWay - Port turnaround time (Load+Discharge) (days)'].iloc[0] *
                                                Costs['Fuel consumption in port (tonnes/day)'].iloc[0] *
                                                Costs['USD/tonne - HFO'].iloc[0]) + \
                                               (Costs['USD/tonne - HFO'].iloc[0] *
                                                Costs['Fuel oil consumption (tonnes/day)'].iloc[0] * 2 * Variables_Prices[
                                                    'Days Travel (one-way inc. transit)'])) /
                                              Costs['Loaded - Gas cargo (mmbtu)'].iloc[0], 0.06)

    # vectorize conditional assignment in pandas dataframe
    # Used nested np.where statements for condtional arguments.
    # https://stackoverflow.com/questions/28896769/vectorize-conditional-assignment-in-pandas-dataframe

    Variables_Prices['CanalCostsOutput'] = np.where(Variables_Prices['Canal Fee'] == 'Panama', (
            Costs['Panama - Total ($)'].iloc[0] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]), \
                                                    np.where(Variables_Prices['Canal Fee'] == 'Suez', (
                                                            Costs['Suez - Total ($)'].iloc[0] /
                                                            Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]), 0))

    Variables_Prices['PortCostsOutput'] = (Variables_Prices['Port Charges'] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0])

    Variables_Prices['TotalCostIncluding(Regas&Port)'] = Variables_Prices['ShippingCostOutput'] + \
                                                         Variables_Prices['BoilOffOutput'] + \
                                                         Variables_Prices['FuelOutput'] + \
                                                         Variables_Prices['CanalCostsOutput'] + \
                                                         Variables_Prices['PortCostsOutput'] + \
                                                         Variables_Prices['Regas Costs']

    Variables_Prices['TotalCostIncluding(Regas)'] = Variables_Prices['ShippingCostOutput'] + \
                                                    Variables_Prices['BoilOffOutput'] + \
                                                    Variables_Prices['FuelOutput'] + \
                                                    Variables_Prices['CanalCostsOutput'] + \
                                                    Variables_Prices['Regas Costs']

    Variables_Prices['TotalCostIncluding(Port)'] = Variables_Prices['ShippingCostOutput'] + \
                                                   Variables_Prices['BoilOffOutput'] + \
                                                   Variables_Prices['FuelOutput'] + \
                                                   Variables_Prices['CanalCostsOutput'] + \
                                                   Variables_Prices['PortCostsOutput']

    Variables_Prices['TotalCost'] = Variables_Prices['ShippingCostOutput'] + \
                                    Variables_Prices['BoilOffOutput'] + \
                                    Variables_Prices['FuelOutput'] + \
                                    Variables_Prices['CanalCostsOutput']
    
    Variables_Prices['TotalCost(Sunk Shiping)'] = Variables_Prices['ShippingCostOutput'] + \
                                Variables_Prices['BoilOffOutput'] + \
                                Variables_Prices['FuelOutput'] + \
                                Variables_Prices['CanalCostsOutput']

    # =================================NETBACK CALCULATION====================================================================
    
    Variables_Prices['Variable_Shipping_Roundtrip'] = Variables_Prices['PointValue_Destination'] - Variables_Prices['BoilOffOutput'] -  Variables_Prices['FuelOutput'] - Variables_Prices['PortCostsOutput'] - Variables_Prices['PointValue_Origin']
    Variables_Prices['Variable_Shipping_Single'] = Variables_Prices['PointValue_Destination'] - 0.5*Variables_Prices['BoilOffOutput'] -  Variables_Prices['FuelOutput'] - Variables_Prices['PortCostsOutput'] - Variables_Prices['PointValue_Origin']
    Variables_Prices['Full_Shipping_Rountrip'] = Variables_Prices['PointValue_Destination'] - Variables_Prices['TotalCostIncluding(Port)'] - Variables_Prices['PointValue_Origin']

    # =========================================================================================================================

    Netback = Variables_Prices[['EffectiveDate', 'PointDate_Origin', 'PointDate_Destination',
                                'RegionFrom', 'TerminalFrom', 'TerminalTo', 'TotalCost', 'TotalCostIncluding(Regas&Port)',
                                'TotalCostIncluding(Regas)',
                                'TotalCostIncluding(Port)','Full_Shipping_Rountrip','Variable_Shipping_Single','Variable_Shipping_Roundtrip']]

    print(dt.strftime('%Y-%m-%d'))
    print(Netback)
    
    Netback2 = Netback2.append(Netback, ignore_index = True )
    Netback2.groupby(['TerminalFrom','TerminalTo'])

    Variables_Prices2 = Variables_Prices2.append(Variables_Prices, ignore_index = True )
    Variables_Prices2.groupby(['TerminalFrom','TerminalTo'])

Netback2.to_csv('netbacktimeseries3.csv')
Variables_Prices2.to_csv('variable_prices_timeseries3.csv')

#===========================================
#SPLITS INTO LIST OF DATAFRAMES 
#https://stackoverflow.com/questions/54046707/pandas-split-one-dataframe-into-multiple-dataframes
#Splits dataframes based on Terminal   
#dfs = dict(tuple(Netback2.groupby(Netback2['TerminalFrom'])))
#columns = ['EffectiveDate', 'PointDate_Origin', 'PointDate_Destination',
       #'TerminalTo', 'Full_Shipping_Rountrip']
#sabine_netback = dfs['Sabine'][columns]
#qatar_netback = dfs['Qatar'][columns]

#====================================================
#Pivoted data
#https://stackoverflow.com/questions/17298313/python-pandas-convert-rows-as-column-headers
#Sabine_Pivot_FullShip = sabine_netback.pivot_table('Full_Shipping_Rountrip', ['PointDate_Destination','EffectiveDate'], 'TerminalTo')
#Sabine_Pivot_FullShip = Sabine_Pivot_FullShip.dropna()
#Sabine_Pivot_FullShip['Netherlands (Gate)-Japan (Tokyo)'] = Sabine_Pivot_FullShip['Netherlands (Gate)'] - Sabine_Pivot_FullShip['Japan (Tokyo)']
#Sabine_Pivot_FullShip['UK (Hook)-Japan (Tokyo)'] = Sabine_Pivot_FullShip['UK (Hook)'] - Sabine_Pivot_FullShip['Japan (Tokyo)']
#=====================================================

#=====================================================
#This removes the entire row if the column UK (Hook)-Japan (Tokyo) is below zero
#https://stackoverflow.com/questions/27759084/how-to-replace-negative-numbers-in-pandas-data-frame-by-zero
#Sabine_Pivot_FullShip = Sabine_Pivot_FullShip[(Sabine_Pivot_FullShip['UK (Hook)-Japan (Tokyo)'] >= 0) | (Sabine_Pivot_FullShip['UK (Hook)'] >= 0)] 
#Replaces values below zero with zero
#Sabine_Pivot_FullShip[Sabine_Pivot_FullShip <= 0] = 0
  
#=====================================================

#Sabine_Pivot_FullShip.to_csv('Sabine_Pivot_FullShip.csv')
#======================================================
#===========================================
#SPLITS INTO LIST OF DATAFRAMES 
#https://stackoverflow.com/questions/54046707/pandas-split-one-dataframe-into-multiple-dataframes
#Splits dataframes based on Terminal   
#dfs = dict(tuple(Netback2.groupby(Netback2['TerminalFrom'])))
#columns = ['EffectiveDate', 'PointDate_Origin', 'PointDate_Destination',
       #'TerminalTo', 'Variable_Shipping_Roundtrip']
#sabine_netback = dfs['Sabine'][columns]
#====================================================
#Pivoted data
#https://stackoverflow.com/questions/17298313/python-pandas-convert-rows-as-column-headers
#Sabine_Pivot_VarShip = sabine_netback.pivot_table('Variable_Shipping_Roundtrip', ['PointDate_Destination','EffectiveDate'], 'TerminalTo')
#Sabine_Pivot_VarShip = Sabine_Pivot_VarShip.dropna()
#Sabine_Pivot_VarShip['Netherlands (Gate)-Japan (Tokyo)'] = Sabine_Pivot_VarShip['Netherlands (Gate)'] - Sabine_Pivot_VarShip['Japan (Tokyo)']
#Sabine_Pivot_VarShip['UK (Hook)-Japan (Tokyo)'] = Sabine_Pivot_VarShip['UK (Hook)']-Sabine_Pivot_VarShip['Japan (Tokyo)']
#Sabine_Pivot_VarShip.to_csv('Sabine_Pivot_VarShip_Orig.csv')

#===================================================

print('Processing by TerminalFrom ...')

columns = ['EffectiveDate', 'PointDate_Origin', 'PointDate_Destination', 'TerminalTo', 'Variable_Shipping_Roundtrip']
netback_by_terminal_from = Netback2[columns].groupby(Netback2['TerminalFrom'])

for terminal_from, group in netback_by_terminal_from:
    print(f'Processing TerminalFrom {terminal_from}')

    pivoted = group.pivot_table(
        'Variable_Shipping_Roundtrip', ['PointDate_Destination', 'EffectiveDate'], 'TerminalTo'
    )
    pivoted.dropna(inplace=True)
    pivoted['Netherlands (Gate)-Japan (Tokyo)'] = pivoted['Netherlands (Gate)'] - pivoted['Japan (Tokyo)']
    pivoted['UK (Hook)-Japan (Tokyo)'] = pivoted['UK (Hook)'] - pivoted['Japan (Tokyo)']

    csv_file = f'{terminal_from}_Pivot_VarShip.csv'
    pivoted.to_csv(csv_file)

    print(f'Data for TerminalFrom {terminal_from} saved to {csv_file}')

print('Processing by TerminalFrom finished')
#==========================================================================
print('Processing by TerminalFrom ...')

columns = ['EffectiveDate', 'PointDate_Origin', 'PointDate_Destination', 'TerminalTo', 'Full_Shipping_Rountrip']
netback_by_terminal_from = Netback2[columns].groupby(Netback2['TerminalFrom'])

for terminal_from, group in netback_by_terminal_from:
    print(f'Processing TerminalFrom {terminal_from}')

    pivoted = group.pivot_table(
        'Full_Shipping_Rountrip', ['PointDate_Destination', 'EffectiveDate'], 'TerminalTo'
    )
    pivoted.dropna(inplace=True)
    pivoted['Netherlands (Gate)-Japan (Tokyo)'] = pivoted['Netherlands (Gate)'] - pivoted['Japan (Tokyo)']
    pivoted['UK (Hook)-Japan (Tokyo)'] = pivoted['UK (Hook)'] - pivoted['Japan (Tokyo)']

    csv_file = f'{terminal_from}_Pivot_FullShip.csv'
    pivoted.to_csv(csv_file)

    print(f'Data for TerminalFrom {terminal_from} saved to {csv_file}')

print('Processing by TerminalFrom finished')

***

In [47]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

2021-01-15 17:11:09,084: 49244 matplotlib.pyplot DEBUG: Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2021-01-15 17:11:09,084: matplotlib.pyplot: 211: DEBUG: Loaded backend module://ipykernel.pylab.backend_inline version unknown..


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import sys, os

sys.path.append(r'C:\git\UAT\gmt-fom-gas-dashboard')
sys.path.append(r"C:\git\CHRIS_SCRIPTS")

# Net Back

In [49]:
# from gmt.dna.amp.models.gas.net_back.net_back import *
# from gmt.dna.amp.models.tests.gas.net_back.test_netback import *
from gmt.fa.lib.models.net_back import *

import nest_asyncio
nest_asyncio.apply()


# Data

In [50]:
data_folder_path = r"C:\git\CHRIS_SCRIPTS"
os.chdir(data_folder_path)
Costs = pd.read_csv(r'Costs.csv')
Variables = pd.read_csv(r'Variables.csv')

In [51]:
Variables['Days Travel temp'] = Variables['Distances']  \
/ (Costs['Laden - Speed (knots)'].iloc[0] * 24)

choices = [0, 1, 1.3]
conditions = [
    (Variables['Canal Fee'] == 'No'),
    (Variables['Canal Fee'] == 'Suez'),
    (Variables['Canal Fee'] == 'Panama')]
Variables['Add days'] = np.select(conditions, choices, default=9999999)

Variables['Days Travel (one-way inc. transit)'] = Variables['Days Travel temp'] \
+ Variables['Add days']

Variables['Days Travel (Return - inc. time at port)'] = \
                    + Variables['Days Travel (one-way inc. transit)'] * 2 \
                    + Costs['OneWay - Port turnaround time (Load+Discharge) (days)'].iloc[0]

Variables['Boil-off'] =  \
    Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]  \
    * Costs['Laden - Natural boil-off gas (%/day)'].iloc[0] \
    * Variables[ 'Days Travel (Return - inc. time at port)'] 

Variables['Delivered Volume'] =   Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]\
                                - Variables['Boil-off']


choices2 = ['EU', 'EU', 'EU', 'ME', 'ME', 'NEA', 'NEA', 'LatAm', 'EU', 'EU', 'EU', 'EU', 'EU']
conditions2 = [
    (Variables['TerminalTo'] == 'UK (Hook)'),
    
    (Variables['TerminalTo'] == 'UK (Hook)(NBP)'),
    
    (Variables['TerminalTo'] == 'Belgium (Zee)'),
    (Variables['TerminalTo'] == 'India (Dhambol)'),
    (Variables['TerminalTo'] == 'Middle East (UAE)'),
    (Variables['TerminalTo'] == 'China (Guandong)'),
    (Variables['TerminalTo'] == 'Japan (Tokyo)'),
    (Variables['TerminalTo'] == "Argentina (B'Blanca)"),
    (Variables['TerminalTo'] == 'France (Montoir)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)'),
    
    (Variables['TerminalTo'] == 'Italy (Rovigo)(PSV)'),
    
    (Variables['TerminalTo'] == 'Netherlands (Gate)')]

Variables['RegionCode'] = np.select(conditions2, choices2, default=9999999)

conditions3 = [
    (Variables['TerminalTo'] == "Argentina (B'Blanca)"),
    (Variables['TerminalTo'] == 'Belgium (Zee)'),
    (Variables['TerminalTo'] == 'China (Guandong)'),
    (Variables['TerminalTo'] == 'France (Montoir)'),
    (Variables['TerminalTo'] == 'India (Dhambol)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)(PSV)'),
    (Variables['TerminalTo'] == 'Japan (Tokyo)'),
    (Variables['TerminalTo'] == 'Middle East (UAE)'),
    (Variables['TerminalTo'] == 'Netherlands (Gate)'),
    (Variables['TerminalTo'] == 'Netherlands (Gate)(TTF)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)(PVB)'),
    (Variables['TerminalTo'] == 'UK (Hook)'),
    (Variables['TerminalTo'] == 'UK (Hook)(NBP)')]

choices3 = ['LNG_H_DES_Lat_Am_MID_M_USD', 
            'G_H_Zeebrugge-Hub_Pnc', 
            'LNG_JKM_M_USD', 
            'G_H_PEG-Nord_EUR',
            'LNG_DES_ME_W_INDIA_M_USD',
            'LNG_H_DES_Med_MID_M_USD',
            "G_H_PSV_EUR", 
            'LNG_JKM_M_USD', 
            'LNG_DES_ME_W_INDIA_M_USD', 
            'LNG_H_DES_NWE_MID_M_USD',
            'G_H_TTF_95%_EUR',
            'LNG_H_DES_Med_MID_M_USD',
            'G_H_AOC_EUR', 
            'LNG_H_DES_NWE_MID_M_USD',
            'G_H_NBP_95%_USD']
Variables['CurveName'] = np.select(conditions3, choices3, default=9999999)

In [52]:
conditions4 = [        
    (Variables['TerminalFrom'] == 'Sabine'),
    (Variables['TerminalFrom'] == 'Cove point'),
    (Variables['TerminalFrom'] == 'Nigeria'),
    (Variables['TerminalFrom'] == 'Angola'),
    (Variables['TerminalFrom'] == 'Qatar'),
    (Variables['TerminalFrom'] == 'Algeria'),
    (Variables['TerminalFrom'] == 'Yamal'),
    (Variables['TerminalFrom'] == 'Wheatstone'),
    (Variables['TerminalFrom'] == 'Gladstone'),
    (Variables['TerminalFrom'] == 'T&T')]
choices4 = ['G_NYM_HH_M_115%_USD', 
            'G_NYM_HH_M_115%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD']    
Variables['OriginPriceCurve'] = np.select(conditions4, choices4, default=9999999)  

***

In [53]:
import sqlalchemy as sa
def query_arc_alchemy(sql):
    # ODBC connection string
    _url ='mssql+pyodbc:///?odbc_connect=DATABASE%3DARCTIMESERIES%3BDRIVER%3D%7BSQL+Server+Native+Client+11.0%7D%3BSERVER%3DARCSQL%3BTrace%3D0%3BTrusted_Connection%3DYes'
    engine =sa.create_engine(_url, echo=False, convert_unicode=True)
    with engine.connect() as connection:
        result =connection.execute(sql)
        data =result.fetchall()
        column_names =result.keys()
        dataframe = pd.DataFrame.from_records(data, columns=column_names)

        return dataframe

    
def daterange(date1, date2):
    for n in range(int((date2 - date1).days) + 1):
        yield date1 + timedelta(n)

In [2]:
def _get_one_arc_curve(curve, _t, _from_date,
                       _to_date) -> pd.DataFrame:
    try:
        data_adaptor = DataAdapterFactory.create("arc", env="prod")
        curve_df = data_adaptor.get_forward_curve(
            curve, _t).resample('M').mean().loc[_from_date:_to_date]
        curve_df.index = [
            date(t.year, t.month, 1) for t in curve_df.index
        ]
        curve_df = curve_df.to_frame()
        curve_df.reset_index(inplace=True)
        curve_df.columns = [
            NetBackSymbols.SYM_POINTDATE, NetBackSymbols.SYM_POINTVALUE
        ]
        curve_df[NetBackSymbols.SYM_CURVENAME] = curve
        return curve_df
    except Exception as e:
        logger.info(e.args[0])
        logger.info(
            f'Issue with arc retrieval for curve={curve}, effective date={_t}'
        )
        return pd.DataFrame()

arc_curves = ['G_H_NBP.USD',
'G_H_Zeebrugge-Hub.Pnc',
'LNG_JKM_M.USD',
'G_H_TRS.EUR',
'LNG_DES_ME_W_INDIA_M.USD',
'LNG_H_DES_Med_MID_M.USD',
'LNG_H_DES_JKM_MID_M.USD',
'LNG_H_DES_NWE_MID_M.USD',
'G_H_TTF.EUR',
'G_H_CEGH.EUR',
'G_H_AOC.EUR',
'G_H_PEG-Nord.EUR',
'G_H_PSV.EUR',
'G_NYM_HH_M.USD',
'LNG_H_DES_Lat_Am_MID_M.USD',
'FX_curve_cross_EUR.USD',
'FX_curve_input_USD.GBP',
'FREIGHT_H_LNG_M_RolledOver.USD']

NameError: name 'pd' is not defined

In [55]:
weekend = [6, 7]

start_dt = date(2021, 1, 14)
end_dt = date(2021, 1, 14)

Netback_all_t = pd.DataFrame()

Netback_short_all_t = pd.DataFrame()

n = len(list(daterange(start_dt, end_dt)))
print(f'There are {n} days to process.\nLeft:', end=' ')

for dt in list(daterange(start_dt, end_dt)):
    print(f'{n-1}', end='; '); n-=1

    if dt.isoweekday() in weekend:
        continue  

    today = dt.day
    if today > 20:
        from_date = dt + pd.DateOffset(months=2)
        to_date = dt + pd.DateOffset(months=2 ,years=3)    
    else:
        from_date = dt + pd.DateOffset(months=1)
        to_date = dt + pd.DateOffset(months=1 ,years=3)


    query_str_1 = f"""
                    SELECT
                      d.[CurveName],
                      datefromparts(year(min(d.PointDate)), month(min(d.PointDate)), 1) as PointDate,
                      AVG(d.[PointValue]) as PointValue
                    FROM [arc_ts_data].[PriceCurveData] d
                    WHERE d.EffectiveDate = '{dt.strftime("%Y-%m-%d")}' and [CurveName] in 
                    ('G_H_NBP.USD', 'G_H_Zeebrugge-Hub.Pnc','LNG_JKM_M.USD', 'G_H_TRS.EUR', 
                    'LNG_DES_ME_W_INDIA_M.USD', 'LNG_H_DES_Med_MID_M.USD', 'LNG_H_DES_JKM_MID_M.USD', 
                    'LNG_H_DES_NWE_MID_M.USD', 'G_H_TTF.EUR', 'G_H_CEGH.EUR', 'G_H_AOC.EUR', 'G_H_PEG-Nord.EUR', 
                    'G_H_PSV.EUR','G_NYM_HH_M.USD' , 'LNG_H_DES_Lat_Am_MID_M.USD', 'FX_curve_cross_EUR.USD', 
                    'FX_curve_input_USD.GBP', 'FREIGHT_H_LNG_M_RolledOver.USD')  
                    and d.PointDate >= '{from_date.strftime("%Y-%m-01")}' and d.PointDate < '{to_date.strftime("%Y-%m-01")}'
                    group by d.CurveId, d.CurveName, d.EffectiveDate, 12 * year(d.PointDate) + month(d.PointDate)
                    order by d.CurveID
                    """
    #------------------------------------------------------Prices---------------------------------------------------------------
    t_start = datetime.now()
    Prices = query_arc_alchemy(query_str_1) 
#     Prices = pd.concat([
#         _get_one_arc_curve(curve, dt, from_date, to_date)
#         for curve in arc_curves
#     ],
#         axis=0,
#         sort=False)        
    t_end = datetime.now()
#     print(f'Took={(t_end-t_start).seconds} seconds to retrieve data from arc')
    
    Prices['CurveName'] = Prices['CurveName'].str.replace('.', '_')    
    Prices['PointDate_Origin'] = pd.to_datetime(Prices['PointDate'])
    Prices['EffectiveDate'] = dt
    Prices['EffectiveDate'] = pd.to_datetime(Prices['EffectiveDate'])
    Prices['PointDate_Destination'] = Prices['PointDate_Origin']    
    
    columns = ['CurveName', 'PointValue', 'PointDate_Origin', 
               'EffectiveDate','PointDate_Destination']
    Prices = Prices[columns]   
    
    #--------------------------------------------------------------------
    mapper = Prices.query('CurveName == "FX_curve_cross_EUR_USD"').set_index(
        ['PointDate_Origin'])['PointValue']
    Prices['FX_curve_cross_EUR_USD'] = Prices['PointDate_Origin'].map(mapper)
    
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("_EUR", case=False),
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') / Prices['FX_curve_cross_EUR_USD'].apply(pd.to_numeric, downcast='float') / 3.412 * 0.95,
                                    Prices['PointValue'])
    
    mapper = Prices.query('CurveName == "FX_curve_input_USD_GBP"').set_index(['PointDate_Origin'])['PointValue']
    Prices['FX_curve_input_USD_GBP'] = Prices['PointDate_Origin'].map(mapper)
    Prices['FX_curve_input_USD_GBP'] = Prices['FX_curve_input_USD_GBP'].fillna(method='ffill')
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("_Pnc", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') / Prices['FX_curve_input_USD_GBP'].apply(pd.to_numeric, downcast='float') / 10, 
                                    Prices['PointValue'])
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_NYM_HH_M_USD", case=False), 
                                Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 1.15, 
                                Prices['PointValue'])
    
    Prices['CurveName'] = np.where(Prices['CurveName'] == 'G_NYM_HH_M_USD',
                                   'G_NYM_HH_M_115%_USD',
                                   Prices['CurveName'])
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_NBP_USD", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0.95, 
                                    Prices['PointValue'])   
    
    Prices['CurveName'] = np.where((Prices['CurveName'] == 'G_H_NBP_USD'),
                                   'G_H_NBP_95%_USD',
                                   Prices['CurveName'])
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_TTF_EUR", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0.95, 
                                    Prices['PointValue'])
    
    Prices['CurveName'] = np.where(
                                (Prices['CurveName'] == 'G_H_TTF_EUR'),
                                'G_H_TTF_95%_EUR',
                                Prices['CurveName'])
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_CEGH_EUR", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0, 
                                    Prices['PointValue'])
    
    Prices['CurveName'] = np.where(Prices['CurveName'] == 'G_H_CEGH_EUR',
                                   'LNG_JKM_M_0%_USD',
                                   Prices['CurveName'])
    
    
    
    
    Prices['PointValue'] = Prices['PointValue'].astype(float)
    Prices['FX_curve_cross_EUR_USD'] = Prices['FX_curve_cross_EUR_USD'].astype(float)
    Prices['FX_curve_input_USD_GBP'] = Prices['FX_curve_input_USD_GBP'].astype(float)
    
    date_rng = pd.date_range(start=dt, freq='MS', periods=12, name='Dates')
    combining_date = []
    for d in date_rng:
        Variables['PointDate_Origin'] = d
        combining_date.append(Variables.copy())  
    vars_ts_df = pd.concat(combining_date)    
    
    

    
    mapper = Prices.query('CurveName == "FREIGHT_H_LNG_M_RolledOver_USD"').set_index(['PointDate_Origin'])['PointValue']
    
    vars_ts_df['Shipping Cost'] = np.where(
        vars_ts_df['Days Travel (Return - inc. time at port)'] > 30,        
        
       (((vars_ts_df['PointDate_Origin'] + pd.DateOffset(months=1)).map(mapper) 
        + vars_ts_df['PointDate_Origin'].map(mapper)) / 2),         
        
        vars_ts_df['PointDate_Origin'].map(mapper)    
    )  
    
    vars_ts_df[['Regas Costs', 'Port Charges', 'Boil-off', 'Delivered Volume', 'Shipping Cost']] = vars_ts_df[
        ['Regas Costs', 'Port Charges', 'Boil-off', 'Delivered Volume', 'Shipping Cost']].apply(pd.to_numeric,
                                                                                                downcast='float')
    
        
    vars_ts_df['ShippingCostOutput'] = (
        vars_ts_df['Shipping Cost'] *
        vars_ts_df['Days Travel (Return - inc. time at port)'] /
        Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]
    )
    
    
    
    
    Netback = vars_ts_df.merge(
        Prices[['PointDate_Origin','CurveName','EffectiveDate']], how='inner', on=['PointDate_Origin', 'CurveName'])
    
    Netback = Netback.merge(Prices[['PointDate_Origin', 'CurveName','PointValue']],
                                              how='inner', 
                                              right_on=['PointDate_Origin', 'CurveName'],
                                              left_on=['PointDate_Origin', 'OriginPriceCurve']                                           
                           )   
    

    Netback = Netback.drop(columns=['CurveName_y'])   
    Netback.rename(columns={'CurveName_x':'CurveName_Destination','PointValue':'PointValue_Origin'}, inplace=True)    
    
    Netback['PointDate_Destination'] = Netback['PointDate_Origin']   
    
    Netback['PointDate_Destination'] = np.where(                        
                    Netback['Days Travel (Return - inc. time at port)'] > 30, 
                    Netback['PointDate_Destination'] + pd.DateOffset(months=1), 
                    Netback['PointDate_Destination']
    )    
    
    Netback = Netback.merge(Prices[['PointDate_Destination','CurveName','PointValue']], 
                                right_on=['PointDate_Destination', 'CurveName'],
                                left_on=['PointDate_Destination', 'CurveName_Destination'])

    Netback.rename(columns={'PointValue':'PointValue_Destination'}, inplace=True)

    Netback = Netback.drop(columns=['CurveName'])
    
    Netback[['Boil-off', 'PointValue_Destination','PointValue_Origin']] = \
        Netback[['Boil-off', 'PointValue_Destination', 'PointValue_Origin']].apply(pd.to_numeric,downcast='float')
    
    Netback['BoilOffOutput'] = (
            Netback['PointValue_Destination'] * Netback['Boil-off'] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0])
    
    Netback['FuelOutput'] = np.where(
        
                            Netback['TerminalFrom'] == 'Yamal', 

                            ((Costs['OneWay - Port turnaround time (Load+Discharge) (days)'].iloc[0] *
                              Costs['Fuel consumption in port (tonnes/day)'].iloc[0] *
                              Costs['USD/tonne - HFO'].iloc[0]) + \
                             (Costs['USD/tonne - HFO'].iloc[0] *
                              Costs['Fuel oil consumption (tonnes/day)'].iloc[0] * 2 * Netback[
                                                    'Days Travel (one-way inc. transit)'])) /
                                              Costs['Loaded - Gas cargo (mmbtu)'].iloc[0], 
        
                            0.06)
    #     cond = Netback['TerminalFrom'] == 'Yamal'
    #     a = Costs['OneWay - Port turnaround time (Load+Discharge) (days)'].iloc[0]
    #     b = Costs['Fuel consumption in port (tonnes/day)'].iloc[0]
    #     c = Costs['USD/tonne - HFO'].iloc[0]
    #     d = Costs['USD/tonne - HFO'].iloc[0] 
    #     e = Costs['Fuel oil consumption (tonnes/day)'].iloc[0]
    #     f = 2
    #     g = Netback['Days Travel (one-way inc. transit)']
    #     h = Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]

    #     Netback['FuelOutput'] = np.where(cond, ((a *b *c) + (d *e * f * g)) / h, 0.06)
    
    
    
    Netback['CanalCostsOutput'] = np.where(
                            Netback['Canal Fee'] == 'Panama', (
                                
                                
                            Costs['Panama - Total ($)'].iloc[0] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]), \
                                                    np.where(Netback['Canal Fee'] == 'Suez', (
                                                            Costs['Suez - Total ($)'].iloc[0] /
                                                            Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]), 
                                                             
                                                             0))
#     cond1 = Netback['Canal Fee'] == 'Panama'
#     a = Costs['Panama - Total ($)'].iloc[0]
#     b = Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]
    
#     cond2 = Netback['Canal Fee'] == 'Suez'
#     c = Costs['Suez - Total ($)'].iloc[0]
#     d = Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]
    
#     Netback['CanalCostsOutput'] = np.where(cond1, (a / b), np.where(cond2, (c /d), 0))


    Netback['PortCostsOutput'] = (Netback['Port Charges'] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0])
    
    Netback['TotalCostIncluding(Regas&Port)'] = Netback['ShippingCostOutput'] + \
                                                         Netback['BoilOffOutput'] + \
                                                         Netback['FuelOutput'] + \
                                                         Netback['CanalCostsOutput'] + \
                                                         Netback['PortCostsOutput'] + \
                                                         Netback['Regas Costs']
    
    
    Netback['TotalCostIncluding(Regas)'] = Netback['ShippingCostOutput'] + \
                                                    Netback['BoilOffOutput'] + \
                                                    Netback['FuelOutput'] + \
                                                    Netback['CanalCostsOutput'] + \
                                                    Netback['Regas Costs']
    
    
    Netback['TotalCostIncluding(Port)'] = Netback['ShippingCostOutput'] + \
                                                   Netback['BoilOffOutput'] + \
                                                   Netback['FuelOutput'] + \
                                                   Netback['CanalCostsOutput'] + \
                                                   Netback['PortCostsOutput']
    
        
    Netback['TotalCost'] = Netback['ShippingCostOutput'] + \
                                    Netback['BoilOffOutput'] + \
                                    Netback['FuelOutput'] + \
                                    Netback['CanalCostsOutput']
    
    Netback['TotalCost(Sunk Shiping)'] = Netback['ShippingCostOutput'] + \
                            Netback['BoilOffOutput'] + \
                            Netback['FuelOutput'] + \
                            Netback['CanalCostsOutput']    
    
    
    Netback['Variable_Shipping_Roundtrip'] = (Netback['PointValue_Destination'] - 
                                              Netback['BoilOffOutput'] - 
                                              Netback['FuelOutput'] - 
                                              Netback['PortCostsOutput'] - 
                                              Netback['PointValue_Origin'])

    Netback['Variable_Shipping_Single'] = (Netback['PointValue_Destination'] - 
                                           0.5*Netback['BoilOffOutput'] -  
                                           Netback['FuelOutput'] - 
                                           Netback['PortCostsOutput'] - 
                                           Netback['PointValue_Origin'])
    
    Netback['Full_Shipping_Rountrip'] = (Netback['PointValue_Destination'] - 
                                         Netback['TotalCostIncluding(Port)'] - 
                                         Netback['PointValue_Origin'])
    
    #--------------------------------------------------------------------------------------------------------
    Netback_all_t = Netback_all_t.append(Netback, ignore_index = True )
      
    mask = ['EffectiveDate', 
            'PointDate_Origin', 
            'PointDate_Destination',
            'RegionFrom', 
            'TerminalFrom', 
            'TerminalTo', 
            'TotalCost', 
            'TotalCostIncluding(Regas&Port)',                    
            'TotalCostIncluding(Regas)',
            'TotalCostIncluding(Port)',
            'Full_Shipping_Rountrip',
            'Variable_Shipping_Single',
            'Variable_Shipping_Roundtrip']
    
    Netback_short = Netback[mask]

    Netback_short_all_t = Netback_short_all_t.append(Netback_short, ignore_index = True )

    

    

There are 1 days to process.
Left: 0; 

In [56]:
Netback_all_t['TerminalTo'].unique()

array(['UK (Hook)', 'Netherlands (Gate)', 'India (Dhambol)',
       'Middle East (UAE)', 'China (Guandong)', 'Japan (Tokyo)',
       "Argentina (B'Blanca)", 'France (Montoir)', 'Spain (Barcelona)',
       'Italy (Rovigo)', 'UK (Hook)(NBP)', 'Netherlands (Gate)(TTF)',
       'Italy (Rovigo)(PSV)', 'Spain (Barcelona)(PVB)'], dtype=object)